In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

# Load data from CSV file
df = pd.read_csv('adjusted_customer_data.csv')

# Encode categorical features
label_encoder = LabelEncoder()
df['DishOrdered'] = label_encoder.fit_transform(df['DishOrdered'])
df['OrderTime'] = label_encoder.fit_transform(df['OrderTime'])

# Fit the encoder on the training data
label_encoder.fit(df['OrderType'])

# Transform both training and test data
df['OrderType'] = label_encoder.transform(df['OrderType'])

# Features and Target
X = df[['DishOrdered', 'OrderType', 'OrderTime']]
y = np.arange(len(X))  # Use a dummy target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the decision tree classifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)  # Use the actual target variable y_train

# Make predictions on the entire dataset
df['predicted_index'] = model.predict(X)

# Generate random deals based on the predicted indices
def generate_random_deal(index):
    return f"Special Deal {index}: 15% on your next order!"

df['random_deal'] = df['predicted_index'].apply(generate_random_deal)


In [1]:

# Function to send an email
def send_email(receiver_email, subject, body):
    sender_email = "awaiskhaleequemkh@gmail.com"  # Replace with your email address
    sender_password = "awaisAWM#2002"  # Replace with your email password

    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()

        try:
            server.login(sender_email, sender_password)
        except smtplib.SMTPAuthenticationError as e:
            print(f"Authentication failed. Make sure your email and password are correct. Error: {e}")
            return

        try:
            server.sendmail(sender_email, receiver_email, message.as_string())
            print(f"Email sent to {receiver_email}")
        except smtplib.SMTPException as e:
            print(f"Error sending email to {receiver_email}: {str(e)}")

# Iterate through the DataFrame and send emails
for index, row in df.iterrows():
    receiver_email = row['Email']  # Replace with the column containing customer emails
    subject = "Special Deal for You!"
    body = row['random_deal']  # Assuming 'random_deal' is the column with the generated deals

    # Print email details for debugging
    print(f"Sending email to: {receiver_email}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")

    try:
        send_email(receiver_email, subject, body)
    except Exception as e:
        print(f"Error sending email to {receiver_email}: {str(e)}")
